In [1]:
# Read and preprocess data
import pandas as pd
import preprocess as pre
df = pd.read_csv('./Data/4017137.csv')
df = pre.convertTime(df)
df = pre.calculateETA(df)
df = pre.encodeTime(df, 5)
df = pre.filterETA(df, 4000)
print(df.head())
print(df.tail())
df = df.dropna()

    Day_sin   Day_cos  Hour  Minute  Second  Vehicle_id  Speed   
0 -0.587785 -0.809017     7      49       0     4017137    0.0  \
1 -0.587785 -0.809017     7      49       0     4017137    0.0   
2 -0.587785 -0.809017     7      49      20     4017137    0.0   
3 -0.587785 -0.809017     7      49      20     4017137    0.0   
4 -0.587785 -0.809017     7      49      40     4017137    0.0   

   Passenger_load  Next_stop   Latitude  Longitude  Heading  Traffic_speed   
0        0.000000    4229596  40.483052 -74.437461     44.0             40  \
1        0.014925    4229596  40.483052 -74.437461     44.0             40   
2        0.014925    4229596  40.483028 -74.437489    182.0             40   
3        0.000000    4229596  40.483028 -74.437489    182.0             40   
4        0.000000    4229596  40.483028 -74.437489    182.0             40   

   Temperature  Windspeed  Precipitation  Humidity  Visibility  Stop_distance   
0         57.1        3.8            0.0        62   

In [11]:
# Split data into dependent and independent variables
X = df.iloc[:, df.columns != 'ETA'].values
y = df.iloc[:, df.columns == 'ETA'].values

In [12]:
# Split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False) # Allocates last 20% of data as test set

In [13]:
# Training thes SVR model on the training set
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)

/var/folders/gj/zg9_2w8d5xb827ljt0tjyb8h0000gn/T/ipykernel_46348/1877506561.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


RandomForestRegressor()

In [14]:
# Predicting the test set results
y_pred = regressor.predict(X_test)

In [15]:
# Determine feature importance
from sklearn.inspection import permutation_importance
results = permutation_importance(regressor, X, y, scoring='neg_mean_squared_error')
importance = results.importances_mean
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.00000
Feature: 1, Score: 0.00000
Feature: 2, Score: 628.97280
Feature: 3, Score: 5129.58422
Feature: 4, Score: 82.56831
Feature: 5, Score: 0.00000
Feature: 6, Score: 1573.76412
Feature: 7, Score: 1482.98035
Feature: 8, Score: 211.03434
Feature: 9, Score: 3674.04359
Feature: 10, Score: 30151.09038
Feature: 11, Score: 1417.95408
Feature: 12, Score: 622.10540
Feature: 13, Score: 3474.18984
Feature: 14, Score: 1336.35103
Feature: 15, Score: 0.00000
Feature: 16, Score: 596.28191
Feature: 17, Score: 7843.36318
Feature: 18, Score: 142825.38322


In [16]:

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
r2 = r2_score(y_test, y_pred)
print('R-squared score:', r2)
mae = mean_absolute_error(y_test, y_pred)
print('Mean absolute error:', mae)
mse = mean_squared_error(y_test, y_pred)
print('Mean squared error:', mse)

R-squared score: 0.8579929969660144
Mean absolute error: 62.36302110817942
Mean squared error: 14716.192161345649
